In [10]:
# Initial Imports
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain


import pandas as pd
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()



True

In [25]:
from langchain.document_loaders import PyPDFLoader

import openai
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_ORGANIZATION'] = os.getenv("OPENAI_ORG")

loader = PyPDFLoader("training_manual.pdf")
pages = loader.load_and_split()


In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# For each page, split the document using a text splitter and then add the resulting docs to the store
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 10)
texts = splitter.split_documents(pages)

    

In [29]:
store = Chroma.from_documents(texts, embedding=OpenAIEmbeddings())

Using embedded DuckDB without persistence: data will be transient
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)).


In [44]:
from langchain.agents.agent_toolkits import (
    VectorStoreToolkit,
    VectorStoreInfo,
    create_vectorstore_agent,
    create_pandas_dataframe_agent,
)
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool, tool
import pandas as pd
from langchain.agents import AgentExecutor, Agent, AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
inventory_df = pd.read_csv('inventory.csv')

vectorstore_info = VectorStoreInfo(
    name = "training_manual",
    description  = "A basic restaurant training manual",
    vectorstore=store
    )

toolkit = VectorStoreToolkit(vectorstore_info = vectorstore_info)

df_agent = create_pandas_dataframe_agent(llm = ChatOpenAI(temperature = 0, model_name = 'gpt-3.5-turbo'), df = inventory_df)

store_agent = create_vectorstore_agent(llm = ChatOpenAI(temperature = 0, model_name = 'gpt-3.5-turbo'), toolkit=toolkit, verbose =  True)

llm = ChatOpenAI(temperature = 0, model_name = 'gpt-3.5-turbo')

memory = ConversationBufferMemory(memory_key = "chat_history", return_messages = True)

tools = [
    Tool.from_function(
        func = df_agent.run,
        name = "Check Inventory",
        description = "Useful if you need to search the inventory"
    ),
    Tool.from_function(
        func = store_agent.run,
        name = "Search Training Manual",
        description = "Useful if you need to search the training manual"
        ),
]

agent_executor = initialize_agent(tools=tools, llm = llm, agent = AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose = True, memory=memory) 
prompt = "What is the proper way to pour wine?"
agent_executor.run(input = prompt)



> Entering new AgentExecutor chain...
{
    "action": "Search Training Manual",
    "action_input": "Proper way to pour wine"
}

> Entering new AgentExecutor chain...
This question seems to be related to restaurant training, so I should use the training_manual tool.
Action: training_manual
Action Input: "What is the proper way to pour wine according to a basic restaurant training manual?"
Observation:  Ensure the glass is perfect with no watermarks, stains, or dishwasher chemicals; it’s smart practice to smell glasses on occasion. If the glass is wet from going through the dishwasher, get a dry one; if no dry glasses are available, ensure there are no chemicals and dry the glass with the microfiber cloth. Ensure you pour to the 5 or 9-ounce mark on the wine glass.
Thought:The answer seems complete, but the user may want sources, so I should use the training_manual_with_sources tool.
Action: training_manual_with_sources
Action Input: "What is the proper way to pour wine according to a

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 4283 tokens (4027 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

In [41]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo'), chain_type="stuff", retriever = vectorstore.as_retriever(), verbose=True)

In [49]:
query = ""
qa.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


'The wine section provides information on the shelf life and storage of wine. Inexpensive wines are sold in a ready-to-drink style and should be consumed as such. Wines with a screw cap should be stored vertically, while wines with a cork that are still in boxes should be stored upside down to keep the cork in contact with the wine. The section also provides tips on pouring wine.'

In [50]:
# We want to create a sample inventory sheet by using a pandas dataframe
# The dataframe will have columns for "liquor_name", "ounces_on_hand", "cost per liter"
# We should use random integers for the "ounces_on_hand" and float for the "cost per liter"
# The dataframe will have 50 rows

spirits = spirits_and_liqueurs = [
    'Whiskey',
    'Vodka',
    'Rum',
    'Tequila',
    'Gin',
    'Brandy',
    'Cognac',
    'Scotch',
    'Bourbon',
    'Liqueur',
    'Absinthe',
    'Sambuca',
    'Rye',
    'Sake',
    'Pisco',
    'Armagnac',
    'Calvados',
    'Aquavit',
    'Mezcal',
    'Amaretto',
    'Triple Sec',
    'Cointreau',
    'Chartreuse',
    'Campari',
    'Midori',
    'Frangelico',
    'Kahlua',
    'Baileys Irish Cream',
    'Drambuie',
    'Chambord',
    'Curaçao',
    'Galliano',
    'Maraschino',
    'Jägermeister',
    'Pernod',
    'Bénédictine',
    'Ouzo',
    'Grappa',
    'Limóncello',
    'Cachaça',
    'Pimm\'s',
    'Amaretto Sour',
    'Irish Coffee',
    'Mojito',
    'Margarita',
    'Cosmopolitan',
    'Old Fashioned',
    'Negroni',
    'Martini',
    'Manhattan',
    'Sazerac',
    'Daiquiri'
]

import random
import pandas as pd

# Populate the dataframe
inventory_df = pd.DataFrame(columns=['liquor_name', 'ounces_on_hand', 'cost_per_liter'])

for i in range(50):
    inventory_df.loc[i] = [random.choice(spirits), random.randint(1, 100), random.uniform(1.0, 10.0)]

In [51]:
# Export the dataframe to a csv file

def export_inventory_to_csv(df):
    df.to_csv('inventory.csv', index=False)
    print('Inventory exported to inventory.csv')

export_inventory_to_csv(inventory_df)

Inventory exported to inventory.csv


In [52]:
from langchain.agents import create_pandas_dataframe_agent

df_agent = create_pandas_dataframe_agent(llm=ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo'), df=inventory_df, verbose=True)

In [53]:
inventory_df

,liquor_name,ounces_on_hand,cost_per_liter
0,Rye,100,1.979134
1,Bénédictine,56,3.078094
2,Rye,35,9.150168
3,Amaretto,86,7.552177
4,Liqueur,79,6.920785
5,Limóncello,28,5.594283
6,Limóncello,39,6.280100
7,Ouzo,97,8.924137
8,Midori,13,3.162891
9,Scotch,5,2.989925


In [63]:
df_agent.run("What is total value of the amount of Pernod on hand?")



> Entering new AgentExecutor chain...
Thought: We need to filter the dataframe to only include rows where the liquor_name is "Pernod", then calculate the total value by multiplying the ounces_on_hand by the cost_per_liter and summing the results.
Action: python_repl_ast
Action Input:
```
pernod_df = df[df['liquor_name'] == 'Pernod']
total_value = (pernod_df['ounces_on_hand'] * pernod_df['cost_per_liter']).sum()
print(total_value)
```693.9799716943078

Observation: None
Thought:We have calculated the total value of Pernod on hand.
Final Answer: The total value of the amount of Pernod on hand is printed in the output of the above code.

> Finished chain.


'The total value of the amount of Pernod on hand is printed in the output of the above code.'

In [55]:
os.environ["WOLFRAM_ALPHA_APPID"] = os.getenv("WOLFRAM_APP_ID")

In [57]:
from langchain.agents import load_tools

tool_names = ["python_repl", "wolfram-alpha", "llm-math", "wikipedia"]
tools = load_tools(tool_names, llm=ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo'))

In [58]:
from langchain.tools import Tool, tool

# Load the df_agent as a tool
tools.append(Tool.from_function(func = df_agent.run, name = "Pandas DataFrame Agent", description = "Useful for when you need to query the inventory dataframe"))


In [61]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(llm=ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo'), tools=tools, verbose=True, agent = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION)

In [62]:
agent.run("What is the total value of the amount of Absinthe I have in my inventory?")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)).




> Entering new AgentExecutor chain...
Thought: I should use the Pandas DataFrame Agent tool to query the inventory dataframe for the total value of Absinthe. 

Action:
```
{
  "action": "Pandas DataFrame Agent",
  "action_input": "inventory['Value'][inventory['Item']=='Absinthe'].sum()"
}
```



> Entering new AgentExecutor chain...
Thought: This question is not related to the given dataframe. We need to define the inventory dataframe first.
Action: Define the inventory dataframe
Action Input: 
```
import pandas as pd
inventory = pd.DataFrame({'Item': ['Absinthe', 'Gin', 'Vodka', 'Rum'], 'Value': [10, 20, 30, 40]})
```
Observation: Define the inventory dataframe is not a valid tool, try another one.
Thought:Oops, I meant to use python_repl_ast instead of defining a new dataframe.
Action: Use python_repl_ast to define the inventory dataframe
Action Input: `import pandas as pd; inventory = pd.DataFrame({'Item': ['Absinthe', 'Gin', 'Vodka', 'Rum'], 'Value': [10, 20, 30, 40]})`
Observati

KeyboardInterrupt: 

In [4]:
# Import the recipes.json file as a pandas dataframe

import pandas as pd
import json

# Create an empty DataFrame
recipes_df = pd.DataFrame()

# Read the JSON file line by line
with open('recipes.json') as file:
    for line in file:
        try:
            # Parse the line as a JSON object
            data = json.loads(line)

            # Add the parsed JSON object as a new row to the DataFrame
            recipes_df = recipes_df.append(data, ignore_index=True)

        except json.JSONDecodeError as e:
            print(f"JSONDecodeError: {e.msg} at line {e.lineno}, column {e.colno} (char {e.pos})")

# Print the resulting DataFrame
print(recipes_df)


C:\Users\sjufa\AppData\Local\Temp\ipykernel_13168\1564507503.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recipes_df = recipes_df.append(data, ignore_index=True)
C:\Users\sjufa\AppData\Local\Temp\ipykernel_13168\1564507503.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recipes_df = recipes_df.append(data, ignore_index=True)
C:\Users\sjufa\AppData\Local\Temp\ipykernel_13168\1564507503.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recipes_df = recipes_df.append(data, ignore_index=True)
C:\Users\sjufa\AppData\Local\Temp\ipykernel_13168\1564507503.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recipes_df = recipes

                              Name  \
0                    Christmas pie   
1     Simmer-&-stir Christmas cake   
2               Christmas cupcakes   
3                   Christmas buns   
4               Christmas cupcakes   
...                            ...   
1612             Flavoured butters   
1613             Crispy Roast Duck   
1614        Smoked salmon kedgeree   
1615             Spicy potted crab   
1616          Sage and Onion Sauce   

                                                    url  \
0     https://www.bbcgoodfood.com/recipes/2793/chris...   
1     https://www.bbcgoodfood.com/recipes/1160/simme...   
2     https://www.bbcgoodfood.com/recipes/72622/chri...   
3     https://www.bbcgoodfood.com/recipes/1803633/ch...   
4     https://www.bbcgoodfood.com/recipes/981634/chr...   
...                                                 ...   
1612  https://www.bbcgoodfood.com/recipes/1911/flavo...   
1613  https://www.bbcgoodfood.com/recipes/3248/crisp...   
1614  https:

C:\Users\sjufa\AppData\Local\Temp\ipykernel_13168\1564507503.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recipes_df = recipes_df.append(data, ignore_index=True)
C:\Users\sjufa\AppData\Local\Temp\ipykernel_13168\1564507503.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recipes_df = recipes_df.append(data, ignore_index=True)
C:\Users\sjufa\AppData\Local\Temp\ipykernel_13168\1564507503.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recipes_df = recipes_df.append(data, ignore_index=True)
C:\Users\sjufa\AppData\Local\Temp\ipykernel_13168\1564507503.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recipes_df = recipes

In [5]:
from langchain.agents import create_pandas_dataframe_agent
from langchain.chat_models  import ChatOpenAI
df_agent = create_pandas_dataframe_agent(llm=ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo'), df=recipes_df, verbose=True)

In [6]:
ingredients = df_agent.run("What are the ingredients in the Christmas cupcakes?")



> Entering new AgentExecutor chain...
Thought: We need to find the row in the dataframe that corresponds to the Christmas cupcakes recipe and then extract the ingredients column.
Action: python_repl_ast
Action Input: df.loc[df['Name'] == 'Christmas cupcakes', 'Ingredients'].values[0]
Observation: ['200g dark muscovado sugar', '175g butter, chopped', '700g luxury mixed dried fruit', '50g glacé cherries', '2 tsp grated fresh root ginger', 'zest and juice 1 orange', '100ml dark rum, brandy or orange juice', '85g/3oz pecannuts, roughly chopped', '3 large eggs, beaten', '85g ground almond', '200g plain flour', '½ tsp baking powder', '1 tsp mixed spice', '1 tsp cinnamon', '400g pack ready-rolled marzipan(we used Dr Oetker)', '4 tbsp warm apricotjam or shredless marmalade', '500g pack fondant icingsugar', 'icing sugar, for dusting', '6 gold and 6 silver muffincases', '6 gold and 6 silver sugared almonds', 'snowflake sprinkles']
Thought:We have found the ingredients for the Christmas cupcake

In [7]:
print(ingredients)

The ingredients for the Christmas cupcakes recipe are: ['200g dark muscovado sugar', '175g butter, chopped', '700g luxury mixed dried fruit', '50g glacé cherries', '2 tsp grated fresh root ginger', 'zest and juice 1 orange', '100ml dark rum, brandy or orange juice', '85g/3oz pecannuts, roughly chopped', '3 large eggs, beaten', '85g ground almond', '200g plain flour', '½ tsp baking powder', '1 tsp mixed spice', '1 tsp cinnamon', '400g pack ready-rolled marzipan(we used Dr Oetker)', '4 tbsp warm apricotjam or shredless marmalade', '500g pack fondant icingsugar', 'icing sugar, for dusting', '6 gold and 6 silver muffincases', '6 gold and 6 silver sugared almonds', 'snowflake sprinkles'].


In [76]:
display(ingredients)

"The ingredients for the Christmas cupcakes recipe are: ['200g dark muscovado sugar', '175g butter, chopped', '700g luxury mixed dried fruit', '50g glacé cherries', '2 tsp grated fresh root ginger', 'zest and juice 1 orange', '100ml dark rum, brandy or orange juice', '85g/3oz pecannuts, roughly chopped', '3 large eggs, beaten', '85g ground almond', '200g plain flour', '½ tsp baking powder', '1 tsp mixed spice', '1 tsp cinnamon', '400g pack ready-rolled marzipan(we used Dr Oetker)', '4 tbsp warm apricotjam or shredless marmalade', '500g pack fondant icingsugar', 'icing sugar, for dusting', '6 gold and 6 silver muffincases', '6 gold and 6 silver sugared almonds', 'snowflake sprinkles']."

In [8]:
recipes_df.head(20)

,Name,url,Description,Author,Ingredients,Method
0,Christmas pie,https://www.bbcgoodfood.com/recipes/2793/chris...,Combine a few key Christmas flavours here to m...,Mary Cadogan,"[2 tbsp olive oil, knob butter, 1 onion, finel...",[Heat oven to 190C/fan 170C/gas 5. Heat 1 tbsp...
1,Simmer-&-stir Christmas cake,https://www.bbcgoodfood.com/recipes/1160/simme...,An easy-to-make alternative to traditional Chr...,Mary Cadogan,"[175g butter, chopped, 200g dark muscovado sug...","[Put the butter, sugar, fruit, zests, juice an..."
2,Christmas cupcakes,https://www.bbcgoodfood.com/recipes/72622/chri...,These beautiful and classy little cakes make l...,Sara Buenfeld,"[200g dark muscovado sugar, 175g butter, chopp...","[Tip the sugar, butter, dried fruit, whole che..."
3,Christmas buns,https://www.bbcgoodfood.com/recipes/1803633/ch...,Paul Hollywood's fruit rolls can be made ahead...,Paul Hollywood,"[500g strong white flour, plus extra for dusti...",[Put the flour and 1 tsp salt into a large bow...
4,Christmas cupcakes,https://www.bbcgoodfood.com/recipes/981634/chr...,"Made these for the second time today, and I ha...",Barney Desmazery,"[280g self-raising flour, 175g golden caster s...",[Heat oven to 190C/170 fan/gas 5 and line a 12...
5,Christmas slaw,https://www.bbcgoodfood.com/recipes/890635/chr...,"A nutty winter salad which is superhealthy, qu...",Good Food,"[2 carrots, halved, ½ white cabbage, shredded,...",[Peel strips from the carrots using a vegetabl...
6,Christmas mess,https://www.bbcgoodfood.com/recipes/2806664/ch...,Delicious and a synch to make! Have made this ...,Caroline Hire,"[600ml double cream, 400g Greek yoghurt, 4 tbs...",[In a small saucepan gently heat the frozen be...
7,Christmas brownies,https://www.bbcgoodfood.com/recipes/christmas-...,can I made these the day before,Miriam Nice,"[200g unsalted buttercut into cubes, plus extr...",[Grease and line a 20cm x 20cm brownie tin. He...
8,Christmas cosmopolitan,https://www.bbcgoodfood.com/recipes/889643/chr...,How many servings have you gotten using the ab...,Good Food,"[500ml vodka, 500ml gingerwine, 1l cranberry j...",[Mix the vodka and ginger wine in a jug. Stir ...
9,Christmas pizza,https://www.bbcgoodfood.com/recipes/christmas-...,Use up leftover roast turkey and sausagemeat s...,Katy Greenwood,"[145g pizza basemix, 6 tbsp tomato pasta sauce...",[Heat oven to 220C/200C fan/gas 7. Prepare the...


In [9]:
df_agent.run("What is the recipe for the Christmas muffin mix?")



> Entering new AgentExecutor chain...
Thought: Look for the row in the dataframe that corresponds to the Christmas muffin recipe
Action: Use boolean indexing to filter the dataframe for the row with the Christmas muffin recipe
Action Input: `df[df['Name'] == 'Christmas cupcakes']`
Observation: Use boolean indexing to filter the dataframe for the row with the Christmas muffin recipe is not a valid tool, try another one.
Thought:The `loc` method can be used to filter the dataframe for the row with the Christmas muffin recipe
Action: Use the `loc` method to filter the dataframe for the row with the Christmas muffin recipe
Action Input: `df.loc[df['Name'] == 'Christmas cupcakes']`
Observation: Use the `loc` method to filter the dataframe for the row with the Christmas muffin recipe is not a valid tool, try another one.
Thought:The `iloc` method can be used to filter the dataframe for the row with the Christmas muffin recipe
Action: Use the `iloc` method to filter the dataframe for the ro

KeyboardInterrupt: 